# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [4]:
data.shape

(68584, 7)

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
#Creo un DF de los datos en crudo que coD y ProductName
df=pd.DataFrame(data.groupby(['CustomerID','ProductName']).Quantity.sum())
df.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
#Mediante pivot table creo un DF con los index por producto y en columna el id del consumidor
data2=pd.pivot_table(df, values='Quantity', index=['ProductName'],columns=['CustomerID'])
data2=data2.fillna(0)
data2=data2.astype(int)
data2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
#Creo una matriz de similitudes con el metodo squareform y pdist, uso euclidiano como metrica
1/(1+squareform(pdist(data2.T, 'euclidean')))

array([[1.        , 0.0774211 , 0.08704668, ..., 0.00435486, 0.00416667,
        0.00433325],
       [0.0774211 , 1.        , 0.07844774, ..., 0.0043666 , 0.00416601,
        0.00433456],
       [0.08704668, 0.07844774, 1.        , ..., 0.00436527, 0.00417944,
        0.00433325],
       ...,
       [0.00435486, 0.0043666 , 0.00436527, ..., 1.        , 0.0032772 ,
        0.00326622],
       [0.00416667, 0.00416601, 0.00417944, ..., 0.0032772 , 1.        ,
        0.0032883 ],
       [0.00433325, 0.00433456, 0.00433325, ..., 0.00326622, 0.0032883 ,
        1.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [11]:
#Creo un DF con las similitudes entre los consumidores, luego escojo el primer consumidor (33), ordeno de menor
#a mayor y agrego los primeros cinco a una variable
dist=pd.DataFrame(1/(1 + squareform(pdist(data2.T, 'euclidean'))),index=data2.columns, columns=data2.columns)
top5similar=dist[33].sort_values(ascending=False)[1:].head()
top5similar





def top5most_sim(i):
    return dist[i].sort_values(ascending=False)[1:].head()


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [12]:
#Una vez teniendo los ID de los cinco consumidores mas similares, obtengo los productos que compraron
top5ind=list(top5similar.index)
data5=df.loc[top5ind]
data5.head()


def top5prod(i):
        return df.loc[list(i.index)]



# Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [13]:
#Luego de tener los productos que compraron esos cinco consumidores, obtengo los que más compraron ellos
datas6=data5.groupby('ProductName').Quantity.sum()
dat6=datas6.sort_values(ascending=False)[1:]
data6=pd.DataFrame(dat6)
data6.head()




def most_buyed(i):
    a=i.groupby('ProductName').Quantity.sum()
    b=a.sort_values(ascending=False)[1:]
    c=pd.DataFrame(b)
    return c



## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [14]:
#Esta lista me da los nombres de los productos que no compraron esos cinco consumidores
notpurchased=[e for e in data2.index if e not in data6.index]

In [15]:
#De esta lista obtengo una lista de tuplas ordenadas de mayor a menor con los productos que no compraron esos
#cinco consumidores, con su precio
top=[(e,data2.loc[e].max()) for e in notpurchased]
top.sort(key=lambda tup:tup[1],reverse=True)
top[:5]




def prodnotbuyed(i):
    productbyconsumer=data2.copy()
    notpurchased=[e for e in productbyconsumer.index if e not in i.index]
    top=[(e,data2.loc[e].max()) for e in notpurchased]
    top.sort(key=lambda tup:tup[1],reverse=True)
    return top[:5]
        


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [16]:
#Creo un diccionario vacio donde pondre las recomendaciones por cliente
#Creo una lista con los 1000 consumidores unicos en total
recommprod4customer={}
customlist=list(data.CustomerID.unique())
len(customlist)

1000

In [17]:
#Paso 4to
#Obtiene los cinco mas similares al consumidor "i"
for i in customlist:
    a=top5most_sim(i)
#Paso 5to
#Obtengo un DF con los productos de los cinco mas similares
    b=top5prod(a)
#Paso 6to
#Obtengo un DF con el ranking de los productos mas comprados por los cinco mas similares
    c=most_buyed(b)
#Paso 7mo
#Obtengo los cinco productos que no ha comprado
    d=prodnotbuyed(c)
#Creacion del diccionario con las top 5 recomendaciones por consumidor
    recommprod4customer[i]=d


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [18]:
datarecomm=pd.DataFrame.from_dict(recommprod4customer).T.reset_index()
dfrecomm=datarecomm.rename(columns={"index":"CustomerID",0:"Top 1",1:"Top 2",2:"Top 3",3:"Top 4",4:"Top 5"})
dfrecomm.head()

,CustomerID,Top 1,Top 2,Top 3,Top 4,Top 5
0,61288,"(Longos - Grilled Salmon With Bbq, 92)","(Yeast Dry - Fermipan, 84)","(Bread - French Baquette, 75)","(Coffee - Irish Cream, 75)","(Scampi Tail, 75)"
1,77352,"(Yeast Dry - Fermipan, 84)","(Coffee - Irish Cream, 75)","(Scampi Tail, 75)","(Spice - Peppercorn Melange, 75)","(Wine - Two Oceans Cabernet, 72)"
2,40094,"(Longos - Grilled Salmon With Bbq, 92)","(Yeast Dry - Fermipan, 84)","(Fenngreek Seed, 75)","(Scampi Tail, 75)","(Spice - Peppercorn Melange, 75)"
3,23548,"(Yeast Dry - Fermipan, 84)","(Bread - French Baquette, 75)","(Coffee - Irish Cream, 75)","(Veal - Eye Of Round, 75)","(Wine - Chablis 2003 Champs, 72)"
4,78981,"(Longos - Grilled Salmon With Bbq, 92)","(Bread - French Baquette, 75)","(Fenngreek Seed, 75)","(Spice - Peppercorn Melange, 75)","(Veal - Eye Of Round, 75)"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [20]:
distcos=pd.DataFrame(1/(1 + squareform(pdist(data2.T,'cosine'))),index=data2.columns, columns=data2.columns)
def top5most_simcos(i):
    return distcos[i].sort_values(ascending=False)[1:].head()

recommprod4customercos={}
#Paso 4to
#Obtiene los cinco mas similares al consumidor "i"
for i in customlist:
    a=top5most_simcos(i)
#Paso 5to
#Obtengo un DF con los productos de los cinco mas similares
    b=top5prod(a)
#Paso 6to
#Obtengo un DF con el ranking de los productos mas comprados por los cinco mas similares
    c=most_buyed(b)
#Paso 7mo
#Obtengo los cinco productos que no ha comprado
    d=prodnotbuyed(c)
#Creacion del diccionario con las top 5 recomendaciones por consumidor
    recommprod4customer[i]=d

datarecomm2=pd.DataFrame.from_dict(recommprod4customer).T.reset_index()
dfrecomm2=datarecomm2.rename(columns={"index":"CustomerID",0:"Top 1",1:"Top 2",2:"Top 3",3:"Top 4",4:"Top 5"})
dfrecomm2.head()

,CustomerID,Top 1,Top 2,Top 3,Top 4,Top 5
0,61288,"(Longos - Grilled Salmon With Bbq, 92)","(Yeast Dry - Fermipan, 84)","(Bread - French Baquette, 75)","(Spice - Peppercorn Melange, 75)","(Veal - Eye Of Round, 75)"
1,77352,"(Longos - Grilled Salmon With Bbq, 92)","(Coffee - Irish Cream, 75)","(Veal - Eye Of Round, 75)","(Garlic - Peeled, 72)","(Wine - Chablis 2003 Champs, 72)"
2,40094,"(Longos - Grilled Salmon With Bbq, 92)","(Yeast Dry - Fermipan, 84)","(Bread - French Baquette, 75)","(Coffee - Irish Cream, 75)","(Spice - Peppercorn Melange, 75)"
3,23548,"(Longos - Grilled Salmon With Bbq, 92)","(Yeast Dry - Fermipan, 84)","(Coffee - Irish Cream, 75)","(Scampi Tail, 75)","(Appetizer - Mini Egg Roll, Shrimp, 72)"
4,78981,"(Longos - Grilled Salmon With Bbq, 92)","(Yeast Dry - Fermipan, 84)","(Bread - French Baquette, 75)","(Scampi Tail, 75)","(Spice - Peppercorn Melange, 75)"
